## Populate an RDF database

This notebook reports the main steps to download CSV files, process them and create an RDF dataset from them accordingly to an ontology. 

To measure execution time in Jupyter notebooks: <code>pip install ipython-autotime</code>

In [132]:
# required libraries
import pandas as pd
import os
from pathlib import Path

In [133]:
# parameters and URLs
path = str(Path(os.path.abspath(os.getcwd())).parent.absolute())

supplyChainUrl = path + '\\HP\\DB2-RDF-Project\\SCMS_Delivery_History_Dataset(ORIGINAL).csv'

# country codes
# countriesURL = path + '/data/countryCodes/wikipedia-iso-country-codes.csv'

# saving folder
savePath =  path + '\\HP\\Desktop\\'

## Project

In [134]:
# Load the CSV files in memory
# we need to convert NaN values to something else otherwise NA strings are converted to NaN
supplyChain = pd.read_csv(supplyChainUrl, sep=',', index_col='ID', keep_default_na=False, na_values=['_'])

supplyChain.dtypes

Project Code                     object
PQ #                             object
PO / SO #                        object
ASN/DN #                         object
Country                          object
Managed By                       object
Fulfill Via                      object
Vendor INCO Term                 object
Shipment Mode                    object
PQ First Sent to Client Date     object
PO Sent to Vendor Date           object
Scheduled Delivery Date          object
Delivered to Client Date         object
Delivery Recorded Date           object
Product Group                    object
Sub Classification               object
Vendor                           object
Item Description                 object
Molecule/Test Type               object
Brand                            object
Dosage                           object
Dosage Form                      object
Unit of Measure (Per Pack)        int64
Line Item Quantity                int64
Line Item Value                 float64


In [135]:
#load the country codes
# we need to convert NaN values to something else otherwise NA strings are converted to NaN -> problem with Namibia
# countries = pd.read_csv(countriesURL, sep=',', index_col='Name', keep_default_na=False, na_values=['_'])

In [136]:
supplyChain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10324 entries, 1 to 86823
Data columns (total 32 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Project Code                  10324 non-null  object 
 1   PQ #                          10324 non-null  object 
 2   PO / SO #                     10324 non-null  object 
 3   ASN/DN #                      10324 non-null  object 
 4   Country                       10324 non-null  object 
 5   Managed By                    10324 non-null  object 
 6   Fulfill Via                   10324 non-null  object 
 7   Vendor INCO Term              10324 non-null  object 
 8   Shipment Mode                 10324 non-null  object 
 9   PQ First Sent to Client Date  10324 non-null  object 
 10  PO Sent to Vendor Date        10324 non-null  object 
 11  Scheduled Delivery Date       10324 non-null  object 
 12  Delivered to Client Date      10324 non-null  object 
 13  D

We need to install <code>RDFLib</code>

<code>pip3 install rdflib </code> [Documentation](https://rdflib.readthedocs.io/en/stable/gettingstarted.html)

In [228]:
import sys
!{sys.executable} -m pip install rdflib
# Load the required libraries
from rdflib import Graph, Literal, RDF, URIRef, Namespace
# rdflib knows about some namespaces, like FOAF
from rdflib.namespace import FOAF, XSD
from urllib.parse import quote



In [186]:
# Construct the country and the supplyChain ontology namespaces not known by RDFlib
# CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
SCO = Namespace("http://www.semanticweb.org/MONC/ontologies/2023/11/supplyChain#")

#create the graph
g = Graph()

# Bind the namespaces to a prefix for more readable output
# g.bind("foaf", FOAF)
# g.bind("countries", CNS)
g.bind("xsd", XSD)
g.bind("sco", SCO)


In [187]:
# CHECK DATE 
import datetime


In [188]:
%%time 
#measure execution time

#iterate over the supplyChain dataframe
for index, row in supplyChain.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the supplyChain id as URI
    idP = "project"+str(index)
    Project = URIRef(SCO[idP])
    
    # Add triples using store's add() method
    g.add((Project, RDF.type, SCO.project))
    g.add((Project, SCO['ManagedBy'], Literal(row['Managed By'], datatype=XSD.string)))
    g.add((Project, SCO['ProjectCode'], Literal(row['Project Code'], datatype=XSD.string)))
    
g.bind("xsd", XSD)
g.bind("sco", SCO)

Wall time: 992 ms


In [189]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'projects.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
Wall time: 1.38 s


# Countries

In [236]:
#create a new graph
g = Graph()

In [237]:
%%time 
#measure execution time

#iterate over the supplyChain dataframe
for index, row in supplyChain.iterrows():
    # Create the node to add to the Graph
    # the node has the namespace + the supplyChain id as URI
    idC = "country"+str(index)
    
    # Using the quote function to escape the special characters
    Country = URIRef(SCO[quote(row['Country'])])
    
    # Add triples using store's add() method.
    g.add((Country, RDF.type, SCO.country))
    g.add((Country, SCO['Country'], Literal(row['Country'], datatype=XSD.string)))
    
g.bind("xsd", XSD)
g.bind("sco", SCO)

Wall time: 767 ms


In [238]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'countries.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
Wall time: 7.39 ms


# Product

In [223]:
#create a new graph
g = Graph()

In [224]:
%%time 
#measure execution time

#iterate over the supplyChain dataframe
for index, row in supplyChain.iterrows():
    
    # Create the node to add to the Graph
    # the node has the namespace + the supplyChain id as URI
    idPr = "product"+str(index)
    Product = URIRef(SCO[idPr])
    
    # Add triples using store's add() method.
    ProductGroup = URIRef(SCO[row['Product Group']])
    g.add((Product, RDF.type, ProductGroup))  
        
    g.add((Product, SCO['Brand'], Literal(row['Brand'], datatype=XSD.string)))
    g.add((Product, SCO['Dosage'], Literal(row['Dosage'], datatype=XSD.string)))
    g.add((Product, SCO['DosageForm'], Literal(row['Dosage Form'], datatype=XSD.string)))
    g.add((Product, SCO['FirstLineDesignation'], Literal(row['First Line Designation'], datatype=XSD.string)))
    
    # Try to cast 'Freight Cost (USD)' to XSD.float
    try:
        freight_literal = Literal(float(row['Freight Cost (USD)']), datatype=XSD.float)
    except ValueError:
    # Handle non-numerical values, set it to None, or choose an appropriate default value
        freight_literal = None
        
    # Add the triple to the graph if the casting was successful
    if freight_literal is not None:
        g.add((Product, SCO['FreightCost'], freight_literal))
    
    g.add((Product, SCO['ItemDescription'], Literal(row['Item Description'], datatype=XSD.string)))
    g.add((Product, SCO['LineItemInsurance'], Literal(row['Line Item Insurance (USD)'], datatype=XSD.string)))
    g.add((Product, SCO['LineItemQuantity'], Literal(row['Line Item Quantity'], datatype=XSD.integer)))
    g.add((Product, SCO['LineItemValue'], Literal(row['Line Item Value'], datatype=XSD.float)))
    g.add((Product, SCO['ManufacturingSite'], Literal(row['Manufacturing Site'], datatype=XSD.string)))
    g.add((Product, SCO['MoleculeTestType'], Literal(row['Molecule/Test Type'], datatype=XSD.string)))
    g.add((Product, SCO['SubClassification'], Literal(row['Sub Classification'], datatype=XSD.string)))
    g.add((Product, SCO['UnitofMeasure'], Literal(row['Unit of Measure (Per Pack)'], datatype=XSD.integer)))
    g.add((Product, SCO['UnitPrice'], Literal(row['Unit Price'], datatype=XSD.float)))
    
    # Try to cast 'Weight (Kilograms)' to XSD.float
    try:
        weight_literal = Literal(float(row['Weight (Kilograms)']), datatype=XSD.float)
    except ValueError:
    # Handle non-numerical values, set it to None, or choose an appropriate default value
        weight_literal = None
        
    # Add the triple to the graph if the casting was successful
    if weight_literal is not None:
        g.add((Product, SCO['Weight'], weight_literal))
        
g.bind("xsd", XSD)
g.bind("sco", SCO)

Wall time: 5.74 s


In [225]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'products.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
Wall time: 6.2 s


# Vendor

In [196]:
#create a new graph
g = Graph()

In [197]:
%%time 
#measure execution time

#iterate over the supplyChain dataframe
for index, row in supplyChain.iterrows():
    idV = "vendor"+str(index)
    Vendor = URIRef(SCO[idV])
    g.add((Vendor, RDF.type, SCO.vendor))
    g.add((Vendor, SCO['Vendor'], Literal(row['Vendor'], datatype=XSD.string)))
    g.add((Vendor, SCO['VendorINCOTerm'], Literal(row['Vendor INCO Term'], datatype=XSD.string)))
    
g.bind("xsd", XSD)
g.bind("sco", SCO)

Wall time: 985 ms


In [198]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'vendors.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
Wall time: 2.63 s


# PQ

In [199]:
#create a new graph
g = Graph()

In [200]:
%%time 
#measure execution time

#iterate over the supplyChain dataframe
for index, row in supplyChain.iterrows():
    idPQ = "pq"+str(index)
    PQ = URIRef(SCO[idPQ])
    g.add((PQ, RDF.type, SCO.PQSupplyChain))
    g.add((PQ, SCO['ASN/DNNumber'], Literal(row['ASN/DN #'], datatype=XSD.string)))
    g.add((PQ, SCO['PO/SONumber'], Literal(row['PO / SO #'], datatype=XSD.string)))
    g.add((PQ, SCO['POSentToVendorDate'], Literal(row['PO Sent to Vendor Date'], datatype=XSD.string)))
    g.add((PQ, SCO['PQFirstSenttoClientDate'], Literal(row['PQ First Sent to Client Date'], datatype=XSD.string)))
    g.add((PQ, SCO['PqNumber'], Literal(row['PQ #'], datatype=XSD.string)))
    #try:
    #    datetime.datetime.strptime(str(row['PO Sent to Vendor Date']), '%Y-%m-%d')
    #    g.add((Movie, MO['POSentToVendorDate'], Literal(row['date_published'], datatype=XSD.date)))
    #except ValueError:
    #    if (len(row['date_published'])==4):
    #        g.add((Movie, MO['releaseDate'], Literal(row['date_published']+"-01-01", datatype=XSD.date)))
    
g.bind("xsd", XSD)
g.bind("sco", SCO)

Wall time: 2.59 s


In [201]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'pq.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
Wall time: 2.59 s


# Shipment

In [202]:
#create a new graph
g = Graph()

In [203]:
%%time 
#measure execution time

#iterate over the supplyChain dataframe
for index, row in supplyChain.iterrows():
    idS = "shipment"+str(index)
    Shipment = URIRef(SCO[idS])
    g.add((Shipment, RDF.type, SCO.shipment))
    g.add((Shipment, SCO['DeliveredToClientDate'], Literal(row['Delivered to Client Date'], datatype=XSD.string)))
    g.add((Shipment, SCO['DeliveryRecordedDate'], Literal(row['Delivery Recorded Date'], datatype=XSD.string)))
    g.add((Shipment, SCO['FulfillVia'], Literal(row['Fulfill Via'], datatype=XSD.string)))
    g.add((Shipment, SCO['ScheduledDeliveryDate'], Literal(row['Scheduled Delivery Date'], datatype=XSD.string)))
    g.add((Shipment, SCO['ShipmentMode'], Literal(row['Shipment Mode'], datatype=XSD.string)))
    
g.bind("xsd", XSD)
g.bind("sco", SCO)

Wall time: 1.8 s


In [204]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'shipment.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
Wall time: 3.52 s


# Project-Country Join

In [205]:
#create a new graph
g = Graph()

In [206]:
%%time 
#measure execution time

#iterate over the supplyChain dataframe
for index, row in supplyChain.iterrows():
    idPrj = "project"+str(index)
    idC = "country"+str(index)
    g.add((URIRef(SCO[idPrj]), SCO['hasCountry'], URIRef(SCO[idC])))
    
g.bind("xsd", XSD)
g.bind("sco", SCO)

Wall time: 402 ms


In [207]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'proj-cnt.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
Wall time: 665 ms


# Project-Product Join

In [208]:
#create a new graph
g = Graph()

In [209]:
%%time 
#measure execution time

#iterate over the supplyChain dataframe
for index, row in supplyChain.iterrows():
    idPrj = "project"+str(index)
    idPrd = "product"+str(index)
    g.add((URIRef(SCO[idPrj]), SCO['hasProduct'], URIRef(SCO[idPrd])))
    
g.bind("xsd", XSD)
g.bind("sco", SCO)

Wall time: 408 ms


In [210]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'proj-prod.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
Wall time: 667 ms


# Project-Vendor Join

In [211]:
#create a new graph
g = Graph()

In [212]:
%%time 
#measure execution time

#iterate over the supplyChain dataframe
for index, row in supplyChain.iterrows():
    idPrj = "project"+str(index)
    idV = "vendor"+str(index)
    g.add((URIRef(SCO[idPrj]), SCO['hasVendor'], URIRef(SCO[idV])))
    
g.bind("xsd", XSD)
g.bind("sco", SCO)

Wall time: 408 ms


In [213]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'proj-ven.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
Wall time: 1.64 s


# Project-PQ Join

In [214]:
#create a new graph
g = Graph()

In [215]:
%%time 
#measure execution time

#iterate over the supplyChain dataframe
for index, row in supplyChain.iterrows():
    idPrj = "project"+str(index)
    idPq = "pq"+str(index)
    g.add((URIRef(SCO[idPrj]), SCO['hasPQ'], URIRef(SCO[idPq])))
    
g.bind("xsd", XSD)
g.bind("sco", SCO)

Wall time: 418 ms


In [216]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'proj-pq.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
Wall time: 682 ms


# Project-Shipment Join

In [217]:
#create a new graph
g = Graph()

In [218]:
%%time 
#measure execution time

#iterate over the supplyChain dataframe
for index, row in supplyChain.iterrows():
    idPrj = "project"+str(index)
    idS = "shipment"+str(index)
    g.add((URIRef(SCO[idPrj]), SCO['hasShipment'], URIRef(SCO[idS])))
    
g.bind("xsd", XSD)
g.bind("sco", SCO)

Wall time: 408 ms


In [219]:
%%time
# print all the data in the Turtle format
print("--- saving serialization ---")
with open(savePath + 'proj-ship.ttl', 'w', encoding="utf-8") as file:
    file.write(g.serialize(format='turtle'))


--- saving serialization ---
Wall time: 755 ms
